In [37]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/university /fall 2024/markov model/markvod_models/generated_data`


Status `~/Documents/university /fall 2024/markov model/markvod_models/generated_data/Project.toml`
⌃ [31c24e10] Distributions v0.25.119
⌃ [e9467ef8] GLMakie v0.11.3
  [429524aa] Optim v1.12.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [38]:
include("dimer_difussion.jl")
include("free_difusion.jl")
include("state_generator.jl")

time_sequence_with_split (generic function with 1 method)

In [39]:

using GLMakie
function animate_particles(p1, p2, filename="particle_animation.mp4")
    n_steps = size(p1, 2)
    
    # Create figure
    fig = Figure(size=(600, 600))
    ax = Axis(fig[1, 1], aspect=DataAspect())
    
    # Calculate axis limits
    all_x = vcat(p1[1,:], p2[1,:])
    all_y = vcat(p1[2,:], p2[2,:])
    
    x_range = maximum(all_x) - minimum(all_x)
    y_range = maximum(all_y) - minimum(all_y)
    
    limits!(ax, -1, 1, -1, 1)
    
    # Create initial points
    point1 = scatter!([p1[1,1]], [p1[2,1]], color=:blue, markersize=10)
    point2 = scatter!([p2[1,1]], [p2[2,1]], color=:red, markersize=10)
    
    # Create animation
    record(fig, filename, 1:n_steps; framerate=10) do frame
        # Update particle positions for current frame
        point1[1] = [p1[1,frame]]
        point1[2] = [p1[2,frame]]
        
        point2[1] = [p2[1,frame]]
        point2[2] = [p2[2,frame]]
    end
    
    return filename
end




function reverse_columns_preserve_size(arr)
    num_rows, num_cols = size(arr)
    result = zeros(eltype(arr), num_rows, num_cols)
    
    for col in 1:num_cols
        result[:, col] = arr[:, num_cols - col + 1]
    end
    
    return result
end


reverse_columns_preserve_size (generic function with 1 method)

In [40]:
function shift(vec, shift_x, shift_y)
    return vec .+ [shift_x, shift_y]
end

shift (generic function with 1 method)

In [41]:
function path_correction(particle_path::Matrix{Float64}, new_destination::Vector{Float64})
    
    if size(particle_path, 1) != 2
        error("Particle path must be a 2×n matrix")
    end
    
    if length(new_destination) != 2
        error("New destination must be a 2-element vector")
    end
    
   
    n = size(particle_path, 2)
    
    
    start_point = particle_path[:, 1]
    original_end = particle_path[:, end]
    

    original_displacement = original_end - start_point
    desired_displacement = new_destination - start_point
    
  
    new_path = copy(particle_path)
    
    
    for i in 2:n
        
        t = (i - 1) / (n - 1)
        
       
        expected_position = start_point + original_displacement * t
        
       
        diffusion_component = particle_path[:, i] - expected_position
        
        
        new_expected_position = start_point + desired_displacement * t
        
       
        new_path[:, i] = new_expected_position + diffusion_component
    end
    
    return new_path
end

path_correction (generic function with 1 method)

# fucntion to generate the segements 

In [42]:
function segments(time_division::Vector, mode::Symbol;  D = 0.01, r = 0.2, box = 1.0, dt = 0.016)

if mode == :full && length(time_division) == 3 # all the sections 

        p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],    D = D, r = r, box = box, dt = dt)

    init_post_r = ([p1_middle[1,end], p1_middle[2,end]],[p2_middle[1,end], p2_middle[2,end]])
    p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[3],  D = D, box = box, dt = dt)

    init_post_l = ([p1_middle[1,1], p1_middle[2,1]], [p2_middle[1,1], p2_middle[2,1]])
    p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l, steps = time_division[1], D = D, box = box, dt = dt)
    p1_left_reversed = reverse_columns_preserve_size(p1_left)
    p2_left_reversed = reverse_columns_preserve_size(p2_left)

    p1 = hcat(p1_left_reversed, p1_middle, p1_right)
    p2 = hcat(p2_left_reversed, p2_middle, p2_right)

    return p1, p2

elseif mode == :right_half && length(time_division) == 2 # middle and right half

    p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)

    init_post_r = ([p1_middle[1,end], p1_middle[2,end]],  [p2_middle[1,end], p2_middle[2,end]])
    p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[2],  D = D, box = box, dt = dt)

    p1 = hcat(p1_middle, p1_right)
    p2 = hcat(p2_middle, p2_right)

    return p1, p2

elseif mode == :middle && length(time_division) == 1 # only middle half 
    p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)
    p1 = hcat(p1_middle)
    p2 = hcat(p2_middle)

    return p1, p2

elseif mode == :left_half && length(time_division) == 2 # middle and left half
    p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],   D = D, r = r, box = box, dt = dt)

    init_post_l = ([p1_middle[1,1], p1_middle[2,1]],  [p2_middle[1,1], p2_middle[2,1]])
    p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l,  steps = time_division[1], D = D, box = box, dt = dt)
    p1_left_reversed = reverse_columns_preserve_size(p1_left)
    p2_left_reversed = reverse_columns_preserve_size(p2_left)

    p1 = hcat(p1_left_reversed, p1_middle)
    p2 = hcat(p2_left_reversed, p2_middle)
    return p1, p2

else
    error("Invalid mode or time division")
end 
end

segments (generic function with 1 method)

In [43]:
p1,p2 = segments([10,40], :right_half, r = 0.01, box = 1.0, dt = 0.016)

([0.18920401094322653 0.16665615827872038 … 0.15108879013571533 0.16966947259757184; 0.2485681995780218 0.2293513159941651 … 0.2435679198244321 0.27400798044899377], [0.1792040312308483 0.15933294839113593 … 0.355177131798385 0.32888293478219194; 0.2485883428653317 0.23616091193976904 … 0.16373613364534698 0.20405560065494086])

# function to select the modes 

In [44]:
function modes_selection(section::Vector )

    size_section = length(section)
    if size_section == 3
        return :full
    elseif size_section == 2
        if section[1].first == 1 && section[2].first == 2
            return :left_half
        elseif section[1].first == 2 && section[2].first == 1
            return :right_half
        else
            error("Invalid section combination")
        end
    elseif size_section == 1
        return :middle
    else
        error("Invalid number of sections")
    end

end 

modes_selection (generic function with 1 method)

In [45]:
k12 = 0.5
k21 = 0.15 
states, steps = simulate_states(k12, k21, 10)
transitions = get_state_transitions(states)
time_in_state = time_sequence(states)
split= time_sequence_with_split(states)



5-element Vector{Any}:
 Any[1 => 11, 2 => 86, 1 => 7]
 Any[1 => 7, 2 => 39, 1 => 3]
 Any[1 => 4, 2 => 18, 1 => 1]
 Any[1 => 2, 2 => 249, 1 => 2]
 Any[1 => 2, 2 => 51, 1 => 1]

In [46]:
modes = modes_selection(split[3])

:full

# extracrt time stamps froms segements

In [47]:
function time_stamps(time_state)

    segments = []
    for i in 1:length(time_state)
        push!(segments, time_state[i].second)
    end


    return segments
end 

time_stamps (generic function with 1 method)

In [48]:
time = time_stamps(split[1])

3-element Vector{Any}:
 11
 86
  7

# function to modify array for path correction 

In [49]:
function path_correction!(particle_path, new_destination::Vector{Float64})
    if size(particle_path, 1) != 2
        error("Particle path must be a 2×n matrix")
    end
    
    if length(new_destination) != 2
        error("New destination must be a 2-element vector")
    end
    
    n = size(particle_path, 2)
    
    start_point = particle_path[:, 1]
    original_end = particle_path[:, end]
    
    original_displacement = original_end - start_point
    desired_displacement = new_destination - start_point
    
    for i in 2:n
        t = (i - 1) / (n - 1)
        
        expected_position = start_point + original_displacement * t
        
        diffusion_component = particle_path[:, i] - expected_position
        
        new_expected_position = start_point + desired_displacement * t
        
        particle_path[:, i] = new_expected_position + diffusion_component
    end
    
    return nothing
end

path_correction! (generic function with 1 method)

# function to genrate the whole simulation

In [50]:
function simulation(k_in ,k12_off,changes)

    k12 = k_in
    k21 = k12_off
    states, steps = simulate_states(k12, k21, changes)
    time_in_state = time_sequence_with_split(states)
    split = time_sequence(states)
    
    
    current_time  = 0 
    Particle_1 = []
    Particle_2 = []

    for i in 1:length(time_in_state)

        section_time_stamps = time_stamps(time_in_state[i])
        modes = modes_selection(section_time_stamps)
        println("section_time_stamps: ", section_time_stamps)
        println("modes: ", modes)
        if i == 1
            p1,p2= segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
            Particle_1 = p1
            Particle_2 = p2
            # println("p1: ", size(p1))
            # println("p2: ", size(p2))
        else 
            p1_temp, p2_temp = segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
            # println("p1_temp: ", size(p1_temp))
            # println("p2_temp: ", size(p2_temp))
            shift_x = -p1_temp[1,1] + Particle_1[1,end]
            shift_y = -p1_temp[2,1] + Particle_1[2,end]
            # println("shift_x: ", shift_x)
            # println("shift_y: ", shift_y)
            p1= shift(p1_temp, shift_x, shift_y)
            p2 = shift(p2_temp, shift_x, shift_y)
            # println("p1: ", size(p1[:,1]))
            # println("p2: ", size(p2[:,1]))
            # println("time_in_state: ", time_in_state[i])
            segment_view =  @view p2[:,1:time_in_state[i][1].second]
        
            path_correction!(segment_view, [Particle_2[1,end], Particle_2[2,end]])
            # println("part_1: ", size(Particle_1))
            # println("part_2: ", size(Particle_2))
            Particle_1 = hcat(Particle_1, p1)
            Particle_2 = hcat(Particle_2, p2)

        end


        
        curren_time = length(Particle_1) 

    end
   
    
    return Particle_1, Particle_2, states, steps, [k12, k21]


end 

simulation (generic function with 1 method)

In [51]:
p1,p2,states, steps,k= simulation(0.5, 0.5, 10) # (k_on,k_off,changes ) changes of states 

section_time_stamps: Any[8, 4, 5]
modes: full
section_time_stamps: Any[6, 12, 1]
modes: full
section_time_stamps: Any[20, 5, 6]
modes: full
section_time_stamps: Any[12, 5, 6]
modes: full


ErrorException: type Int64 has no field first

In [52]:

animate_particles(p1, p2, "total_animation.mp4")



"total_animation.mp4"

# implementation on the forward algorithm

In [54]:
function compute_free_density(particle1::Matrix{Float64}, particle2::Matrix{Float64}, index::Int; sigma=0.1, dt=0.01)


# Calculate relative positions for current frame
Δxn = particle1[1, index] - particle2[1, index]  
Δyn = particle1[2, index] - particle2[2, index]  

# Calculate relative positions for previous frame
Δxn_1 = particle1[1, index-1] - particle2[1, index-1]  
Δyn_1 = particle1[2, index-1] - particle2[2, index-1]  

# Calculate squared distances and distances
dn_1square = (Δxn_1^2) + (Δyn_1^2)
dn_1 = sqrt(dn_1square)
dn_square = (Δxn^2) + (Δyn^2)
dn = sqrt(dn_square)

# Calculate density using modified Bessel function
density_val = (dn/sigma^2) * (exp((-(dn_square) - (dn_1square))/sigma^2)) * modified_bessel(dt, dn, dn_1, sigma)

# Handle edge cases with numerical integration
if density_val == Inf || isnan(density_val) || density_val == 0
density_val = 0
for θ in 0:dt:2*pi
density_val += exp(-((dn_square) + (dn_1square) - (2*dn*dn_1*cos(θ)))/(2*sigma^2))
end 
density_val *= (dn/sigma^2)*dt
end

return density_val
end

compute_free_density (generic function with 1 method)

In [55]:
function forward_algorithm(particle1::Matrix{Float64}, particle2::Matrix{Float64}, d_dimer::Float64, param::Vector{Float64}; dt=0.01, sigma=0.1)

    N = min(size(particle1, 2), size(particle2, 2)) - 1
 
    alpha = zeros(2, N)
    scale = zeros(N)
    e1_vec =[]
    e2_vec = []
    # Ensure we don't go out of bounds when calculating initial state probabilities
    frame_index = min(2, N)
    

    e1_initial = compute_free_density(particle1, particle2, frame_index, sigma=sigma, dt=dt)
    
    e2_initial = compute_dimer_density(particle1, particle2, frame_index, d_dimer, sigma=sigma, dt=dt)
    push!(e1_vec, e1_initial)
    push!(e2_vec, e2_initial)
    alpha[1, 1] = e1_initial
    alpha[2, 1] = e2_initial
    
    scale[1] = sum(alpha[:, 1])
    alpha[:, 1] ./= max(scale[1], eps()) # Avoid division by zero
    
   
    k_on = param[1]
    k_off = param[2]
    T = [1-(k_on*dt) k_on*dt; 
         k_off*dt 1-(k_off*dt)]
    
    
    for t in 2:N
       
        frame_index = min(t + 2, size(particle1, 2))
        
        
        e1 = compute_free_density(particle1, particle2, frame_index, sigma=sigma, dt=dt)
        e2 = compute_dimer_density(particle1, particle2, frame_index, d_dimer, sigma=sigma, dt=dt)
        push!(e1_vec, e1)
        push!(e2_vec, e2)
        
        alpha[1, t] = (alpha[1, t-1]*T[1,1] + alpha[2, t-1]*T[2,1]) * e1
        alpha[2, t] = (alpha[1, t-1]*T[1,2] + alpha[2, t-1]*T[2,2]) * e2
        
        
        scale[t] = sum(alpha[:, t])
       # println("scale[t]: ", scale[t])
        alpha[:, t] ./= max(scale[t], eps()) 
    end
    
    
    loglikelihood = sum(log.(max.(scale, eps()))) 
    
    return alpha, loglikelihood, e1_vec, e2_vec
end

forward_algorithm (generic function with 1 method)

In [56]:
function compute_dimer_density(particle1::Matrix{Float64}, particle2::Matrix{Float64},index::Int, d_dimer::Float64; sigma=0.1, dt=0.01)

Δxn = particle1[1, index] - particle2[1, index]  
Δyn = particle1[2, index] - particle2[2, index]  


dn_square = (Δxn^2) + (Δyn^2)
dn = sqrt(dn_square)


density_val = (dn/sigma^2) * exp(((-(d_dimer^2) - (dn_square))/sigma^2)) * modified_bessel(dt, d_dimer, dn, sigma)


if density_val == Inf || isnan(density_val) || density_val == 0
density_val = 0
for θ in 0:dt:2*pi
density_val += exp(-((dn_square) + (d_dimer^2) - (2*dn*d_dimer*cos(θ)))/(2*sigma^2))
end 
density_val *= (dn/sigma^2)*dt
end

return density_val
end

compute_dimer_density (generic function with 1 method)

In [57]:
function modified_bessel(dt, d1, d2, σ)
    result_me = 0.0

    x = (d1*d2)/(σ^2)

    for θ in 0:dt:2*pi
        result_me += exp(x * cos(θ))
    end

    result_me  *= dt / (2*pi)

    return result_me 
end

modified_bessel (generic function with 1 method)

In [58]:
p1,p2 = simulation(0.5, 0.5, 10)

section_time_stamps: Any[2, 22, 9]
modes: full
section_time_stamps: Any[10, 59, 1]
modes: full
section_time_stamps: Any[1, 6, 10]
modes: full
section_time_stamps: Any[10, 11, 21]
modes: full
section_time_stamps: Any[22, 13, 1]
modes: full


([0.9631439311621195 0.9773202970344602 … 0.8420946378798235 0.8420946378798235; 0.5251787602449255 0.542767047192076 … 0.8324010927243005 0.8324010927243005], [0.9804955901151899 0.9681213379900971 … 0.8376985272942873 0.8376985272942873; 0.518734000969482 0.5388454209350074 … 0.8413829751394591 0.8413829751394591], [1, 1, 2, 2, 2, 2, 2, 2, 2, 2  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 1], 197, [0.5, 0.5])

In [59]:
p1

2×198 Matrix{Float64}:
 0.963144  0.97732   0.97732   0.980017  …  0.842095  0.842095  0.842095
 0.525179  0.542767  0.542767  0.56371      0.841038  0.832401  0.832401

In [60]:
σ = 0.01
p1_noise = p1 .+ σ*randn(size(p1))
p2_noise = p2 .+ σ*randn(size(p2))


2×198 Matrix{Float64}:
 0.969922  0.969928  0.978423  1.00331   …  0.84701   0.828145  0.83277
 0.521028  0.549566  0.524799  0.581763     0.859488  0.834952  0.845505

In [61]:
animate_particles(p1_noise, p2_noise, "noisy_animation.mp4")

"noisy_animation.mp4"

In [62]:
alpha, loglikelihood,e1,e2 = forward_algorithm(p1, p2, 0.01, [0.5, 0.5])
alpha_noise, loglikelihood_noise,e1_noise,e2_noise = forward_algorithm(p1_noise, p2_noise, 0.01, [0.5, 0.5])

([0.499353182605938 0.4962150281715143 … 0.0491297356802894 0.049179992277330345; 0.500646817394062 0.5037849718284858 … 0.9508702643197107 0.9508200077226697], 138.44540840742283, Any[1.521632063646827, 2.302928328677802, 1.1138718278778397, 0.9475990767711123, 0.678219349071898, 2.3080328583594043, 0.6085283807276637, 2.6564026808281365, 1.8877133293548398, 1.8951296506712108  …  2.8949399244098704, 0.9920531088667751, 2.828820673953574, 1.3235595495212247, 1.6704342221843387, 2.6958380012867904, 0.8167909070046717, 2.8652419721579307, 1.3705653470682972, 1.3705653470682972], Any[1.5255740354632197, 2.332079310918011, 1.1739516051172445, 0.9518030778134458, 0.6778108733931639, 2.2960839707130436, 0.6400440862342044, 2.6414050271500256, 2.030828933722462, 1.9638284567251858  …  3.0453207876445214, 1.106890189942407, 2.8365262108935467, 1.447398122271484, 1.6909234066226408, 2.7515315333630586, 0.8870676178975967, 2.860067235730738, 1.5018571807705383, 1.5018571807705383])

In [63]:
states_round =round.(alpha[1,:])
states_noise_round = round.(alpha_noise[1,:])

197-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [64]:
using CairoMakie
fig = Figure(size=(800, 600))
ax = Axis(fig[1, 1], title="Free density", xlabel="Time", ylabel="probaility")
ax1 = Axis(fig[2, 1], title="Dimer density", xlabel="Time", ylabel="probaility")
#ax2 = Axis(fig[3, 1], title="position_diference in x ", xlabel="Time", ylabel="distacne")
#ax3 = Axis(fig[4, 1], title="position_diference in y ", xlabel="Time", ylabel="distacne")
ax4 = Axis(fig[6, 1], title="alpha[2,:]", xlabel="Time", ylabel="dimer probaility")
ax5 = Axis(fig[5, 1], title="alpha[1,:]", xlabel="Time", ylabel="free probability")

lines!(ax, e1, color=:blue, label="Free density")
lines!(ax1, e2, color=:red, label="Dimer density")
#lines!(ax2, p1[1,:] - p2[1,:], color=:green, label="x")
#lines!(ax3, p1[2,:] - p2[2,:], color=:orange, label="y")
lines!(ax4, alpha[1,:], color=:purple, label="dimer")
lines!(ax5, alpha[2,:], color=:black, label="free")
axislegend(ax, position = :rt) 
display(fig)


GLMakie.Screen(...)

In [65]:
save("probability_fig.png", fig)

In [66]:
fig2 = Figure(size=(800, 600))
ax6 = Axis(fig2[1, 1], title="Free density with noise", xlabel="Time", ylabel="probaility")
ax7 = Axis(fig2[2, 1], title="Dimer density with noise", xlabel="Time", ylabel="probaility") 
ax8 = Axis(fig2[3, 1], title="alpha_noise[1,:]", xlabel="Time", ylabel="free probaility")
ax9 = Axis(fig2[4, 1], title="alpha_noise[2,:]", xlabel="Time", ylabel="dimer probaility")
lines!(ax6, e1_noise, color=:blue, label="Free density with noise")
lines!(ax7, e2_noise, color=:red, label="Dimer density with noise")
lines!(ax8, alpha_noise[1,:], color=:black, label="free with noise")
lines!(ax9, alpha_noise[2,:], color=:purple, label="dimer with noise")
axislegend(ax6, position = :rt)
display(fig2)   

GLMakie.Screen(...)

In [67]:
save("probability_noise.png", fig2)

In [68]:
using Optim

function objective_function(param, p1, p2, d_dimer)
    _, loglikelihood,e1,e2 = forward_algorithm(p1, p2, d_dimer, param)
    return -loglikelihood
end


initial_params = [0.1, 0.1]


function wrapper(param)
    return objective_function(param, p1, p2, 0.01)
end
function wrapper_noise(param)
    return objective_function(param, p1_noise, p2_noise, 0.01)
end

result = optimize(wrapper, initial_params, NelderMead())
result_noise = optimize(wrapper_noise, initial_params, NelderMead())



optimal_params = Optim.minimizer(result)
optimal_params_noise = Optim.minimizer(result_noise)
k_on_optimized = optimal_params[1]
k_off_optimized = optimal_params[2]
k_off_optimized_noise = optimal_params_noise[2]
k_on_optimized_noise = optimal_params_noise[1]


_, final_loglikelihood,e1,e2 = forward_algorithm(p1, p2, 0.01, optimal_params)
_, final_loglikelihood_noise,e1_noise,e2_noise = forward_algorithm(p1_noise, p2_noise, 0.01, optimal_params_noise)


([0.499353182605938 -79.00002510039849 … -10.439075366839841 -10.439075366839829; 0.500646817394062 80.00002510039849 … 11.439075366839841 11.439075366839829], 41306.46371686692, Any[1.521632063646827, 2.302928328677802, 1.1138718278778397, 0.9475990767711123, 0.678219349071898, 2.3080328583594043, 0.6085283807276637, 2.6564026808281365, 1.8877133293548398, 1.8951296506712108  …  2.8949399244098704, 0.9920531088667751, 2.828820673953574, 1.3235595495212247, 1.6704342221843387, 2.6958380012867904, 0.8167909070046717, 2.8652419721579307, 1.3705653470682972, 1.3705653470682972], Any[1.5255740354632197, 2.332079310918011, 1.1739516051172445, 0.9518030778134458, 0.6778108733931639, 2.2960839707130436, 0.6400440862342044, 2.6414050271500256, 2.030828933722462, 1.9638284567251858  …  3.0453207876445214, 1.106890189942407, 2.8365262108935467, 1.447398122271484, 1.6909234066226408, 2.7515315333630586, 0.8870676178975967, 2.860067235730738, 1.5018571807705383, 1.5018571807705383])

In [69]:
println("Optimized k_on: ", k_on_optimized)
println("Optimized k_off: ", k_off_optimized)
println("Final Log-Likelihood: ", final_loglikelihood)

println("Optimized k_on with noise: ", k_on_optimized_noise)
println("Optimized k_off with noise: ", k_off_optimized_noise)
println("Final Log-Likelihood with noise: ", final_loglikelihood_noise)

Optimized k_on: 8586.773241686766
Optimized k_off: -10195.391679322804
Final Log-Likelihood: 681.5213289818108
Optimized k_on with noise: 5.573055016171961e93
Optimized k_off with noise: -6.610427164207669e93
Final Log-Likelihood with noise: 41306.46371686692


In [73]:
for_val= zeros( (101, 101) )

for i in 0:.01:1
    n=1
    for j in 0:.01:1
        m=1
        alpha, loglikelihood, e1_vec, e2_vec = forward_algorithm(p1, p2, 0.01, [i, j])
        for_val[n,m] = loglikelihood
    end

end 
    

In [80]:
for_val_noise= zeros( (101, 101) )

for i in 0:.01:1
    n=1
    for j in 0:.01:1
        m=1
        alpha, loglikelihood, e1_vec, e2_vec = forward_algorithm(p1_noise, p2_noise, 0.01, [i, j])
        for_val_noise[n,m] = loglikelihood
    end

end 
    

In [ ]:
using CairoMakie

param1_range = 0:0.01:1
param2_range = 0:0.01:1

fig = Figure(size = (1200, 500))

ax1 = Axis(fig[1, 1], 
    xlabel = "Parameter 1 (k_on)", 
    ylabel = "Log-Likelihood",
    title = "Log-Likelihood vs Parameter 1")

ll_param1 = [maximum(for_val[i, :]) for i in 1:101]
lines!(ax1, param1_range, ll_param1, linewidth = 3, color = :blue)

ax2 = Axis(fig[1, 2], 
    xlabel = "Parameter 2 (k_off)", 
    ylabel = "Log-Likelihood",
    title = "Log-Likelihood vs Parameter 2")

ll_param2 = [maximum(for_val[:, j]) for j in 1:101]
lines!(ax2, param2_range, ll_param2, linewidth = 3, color = :red)

fig

In [84]:
save( "likelihood_plots.png",fig)

In [ ]:
using CairoMakie

param1_range = 0:0.01:1
param2_range = 0:0.01:1

fig_noise = Figure(size = (1200, 500))

ax1 = Axis(fig_noise[1, 1], 
    xlabel = "Parameter 1 (k_on)", 
    ylabel = "Log-Likelihood",
    title = "Log-Likelihood vs Parameter 1 with noise ")

ll_param1 = [maximum(for_val_noise[i, :]) for i in 1:101]
lines!(ax1, param1_range, ll_param1, linewidth = 3, color = :blue)

ax2 = Axis(fig_noise[1, 2], 
    xlabel = "Parameter 2 (k_off)", 
    ylabel = "Log-Likelihood",
    title = "Log-Likelihood vs Parameter 2 with noise")


ll_param2 = [maximum(for_val_noise[:, j]) for j in 1:101]
lines!(ax2, param2_range, ll_param2, linewidth = 3, color = :red)

fig_noise

In [86]:
save( "likelihood_plots_noise .png",fig_noise)

In [ ]:
# Create custom time vector for simulation output
function create_time_vector(p1, dt=0.016)
    n_steps = size(p1, 2)
    time_vec = collect(0:dt:(n_steps-1)*dt)
    return time_vec
end

# Generate time vector for the simulation
time_simulation = create_time_vector(p1, 0.016)
println("Time vector length: ", length(time_simulation))
println("First 10 time points: ", time_simulation[1:10])
println("Total simulation time: ", time_simulation[end], " seconds")